In [76]:
import pandas as pd
from scipy.stats import chi2_contingency
import numpy as np
from scipy.stats.contingency import odds_ratio

# Путь к файлу
file_path = 'ЭР.xlsm'

# Чтение данных из файла Excel
try:
    # Попытка загрузить файл
    genetic_data = pd.read_excel(file_path)
except Exception as e:
    # В случае ошибки при чтении файла, сохранить сообщение об ошибке
    error_message = str(e)
    genetic_data = None

# Вывод первых нескольких строк данных для проверки содержимого или сообщения об ошибке
genetic_data.head() if genetic_data is not None else error_message

,№ п\п,Диагноз,Тяжесть заболевание,Вегетативно-сосудистая_полинейропатия,Пояснично-крестцовая радикулопатия,Периферический ангиодистонический синдром,Шейно-плечевая радикулопатия,Гипертоническая болезнь,Ишемическая болезнь,ЦВЗ,...,Эритроциты,СОЭ,САД,ДАД,ЧСС,rs4880 гена SOD2,rs361525 гена TNFA,rs16944 гена IL1B,rs1799750 гена MMP1,rs1800795 гена IL6
0,1,VibrationDisease,0,True,False,False,False,True,False,False,...,5.60,5,110,70,77,3,2,2,1,1
1,2,VibrationDisease,2,False,False,False,False,True,False,False,...,5.19,5,140,90,75,2,3,2,1,1
2,3,VibrationDisease,1,False,False,False,False,False,False,False,...,5.20,3,130,80,66,1,3,3,1,1
3,4,VibrationDisease,2,False,False,True,False,True,True,False,...,4.71,4,120,85,66,2,3,1,2,1
4,5,VibrationDisease,1,True,False,True,False,True,False,False,...,4.75,6,125,80,63,3,3,1,2,1


In [77]:
genetic_data.columns

Index(['№ п\п', 'Диагноз', 'Тяжесть заболевание',
       'Вегетативно-сосудистая_полинейропатия',
       'Пояснично-крестцовая радикулопатия',
       'Периферический ангиодистонический синдром',
       'Шейно-плечевая радикулопатия', 'Гипертоническая болезнь',
       'Ишемическая болезнь', 'ЦВЗ', 'Остеоартроз', 'БА', 'ХОБЛ',
       'Нейросенсорная тугоухость', 'СД', 'Пептическая язва', 'Лейкоциты',
       'Эритроциты', 'СОЭ', 'САД', 'ДАД', 'ЧСС', 'rs4880 гена SOD2',
       'rs361525 гена TNFA', 'rs16944 гена IL1B', 'rs1799750 гена MMP1',
       'rs1800795 гена IL6'],
      dtype='object')

In [78]:
# Задаем соответствия для каждого столбца
mapping_rs4880 = {1: 'CC', 2: 'TC', 3: 'TT'}
mapping_rs361525 = {1: 'AA', 2: 'GA', 3: 'GG'}
mapping_rs16944 = {1: 'AA', 2: 'GA', 3: 'GG'}
mapping_rs1799750 = {1: 'GG', 2: 'Gd', 3: 'dd'}
mapping_rs1800795 = {1: 'CC', 2: 'CG', 3: 'GG'}

# Применяем соответствия к данным
genetic_data['rs4880 гена SOD2'] = genetic_data['rs4880 гена SOD2'].map(mapping_rs4880)
genetic_data['rs361525 гена TNFA'] = genetic_data['rs361525 гена TNFA'].map(mapping_rs361525)
genetic_data['rs16944 гена IL1B'] = genetic_data['rs16944 гена IL1B'].map(mapping_rs16944)
genetic_data['rs1799750 гена MMP1'] = genetic_data['rs1799750 гена MMP1'].map(mapping_rs1799750)
genetic_data['rs1800795 гена IL6'] = genetic_data['rs1800795 гена IL6'].map(mapping_rs1800795)

# Проверяем результат преобразования, выводя первые несколько строк обновленного датафрейма
genetic_data[['rs4880 гена SOD2', 'rs361525 гена TNFA', 'rs16944 гена IL1B', 'rs1799750 гена MMP1', 'rs1800795 гена IL6']].head()


,rs4880 гена SOD2,rs361525 гена TNFA,rs16944 гена IL1B,rs1799750 гена MMP1,rs1800795 гена IL6
0,TT,GA,GA,GG,CC
1,TC,GG,GA,GG,CC
2,CC,GG,GG,GG,CC
3,TC,GG,AA,Gd,CC
4,TT,GG,AA,Gd,CC


In [79]:
def odds_ratio_ci(table, alpha=0.05):
    # Расчет отношения шансов
    or_value = round((table[0, 0] * table[1, 1]) / (table[0, 1] * table[1, 0]), 2)
    
    # Расчет стандартной ошибки логарифма отношения шансов
    se_log_or = np.sqrt(1/table[0, 0] + 1/table[0, 1] + 1/table[1, 0] + 1/table[1, 1])
    
    # Доверительный интервал для логарифма отношения шансов
    ci_log_or = [np.log(or_value) - se_log_or * 1.96, np.log(or_value) + se_log_or * 1.96]
    
    # Возврат к оригинальному масштабу отношения шансов
    ci_or = [round((np.exp(ci_log_or[0])),2), round((np.exp(ci_log_or[1])),2)]
    
    return or_value, ci_or

def calculate_extended_statistics(genetic_data, genotype_col, feature_col):
    # Получаем уникальные генотипы
    genotypes = genetic_data[genotype_col].unique()
    # Общее количество случаев
    total_count = genetic_data.shape[0]
    
    # Результаты для каждого генотипа
    results = []

    for genotype in genotypes:
        # Таблица сопряженности для данного генотипа
        feature_positive = genetic_data[genetic_data[feature_col] == True][genotype_col].value_counts()
        feature_negative = genetic_data[genetic_data[feature_col] == False][genotype_col].value_counts()
        
        
        # Количество случаев с признаком и без него для данного генотипа
        count_positive = feature_positive.get(genotype, 0)
        count_negative = feature_negative.get(genotype, 0)
        
        # Проценты для каждого генотипа
        percent_positive = round(((count_positive / feature_positive.sum()) * 100),2)
        percent_negative = round(((count_negative / feature_negative.sum()) * 100), 2)
        
        # Создание таблицы сопряженности
        contingency_table = np.array([[count_positive, count_negative],
                                      [feature_positive.sum() - count_positive, 
                                       feature_negative.sum() - count_negative]])
        
        # Расчет Хи-квадрат и p-значения
        chi2, p, _, _ = chi2_contingency(contingency_table, correction=True)
        chi2 = round(chi2, 2)
        p = round(p, 2)
        # Расчет OR и 95% CI
        or_value, ci_or = odds_ratio_ci(contingency_table)
        
        # Добавление результатов в список
        results.append({
            'Genotype': genotype,
            'With_Feature': count_positive,
            'With_Feature_Percent': percent_positive,
            'Without_Feature': count_negative,
            'Without_Feature_Percent': percent_negative,
            'Chi2': chi2,
            'p-value': p,
            'Odds Ratio': or_value,
            '95% CI': ci_or
        })
    
    # Создание DataFrame из результатов
    results_df = pd.DataFrame(results)
    
    return results_df



In [80]:
from docx import Document
from docx.oxml.ns import nsdecls
from docx.oxml import parse_xml

def round_confidence_interval(ci):
    return (round(ci[0], 2), round(ci[1], 2))

def create_description(statistics_df, feature_name):
    descriptions = []
    for _, row in statistics_df.iterrows():
        genotype = row['Genotype']
        with_feature_percent = row['With_Feature_Percent']
        without_feature_percent = row['Without_Feature_Percent']
        p_value = row['p-value']
        orf = row['Odds Ratio']
        ci = round_confidence_interval(row['95% CI'])
        description = (
            f"В группе больных с признаком {feature_name} генотип '{genotype}' "
            f"встречается с частотой {with_feature_percent:.2f}%, что "
        )
        if p_value < 0.05:
            description += (
                f"статистически значимо чаще (p={p_value:.3f}), чем в группе без признака, "
                f"где этот генотип встречается с частотой {without_feature_percent:.2f}%. "
                f"Отношения шансов для генотипа момтавляет: {orf}."
            )
        else:
            description += (
                f"статистически не отличается (p={p_value:.3f}) от группы без признака, "
                f"где этот генотип встречается с частотой {without_feature_percent:.2f}%. "
                f"Отношения шансов для генотипа момтавляет: {orf}."
            )
        descriptions.append(description)
    return " ".join(descriptions)



In [81]:
def calculate_allele_statistics(genetic_data, genotype_col, feature_col):
    # Получаем уникальные генотипы и определяем аллели
    genotypes = genetic_data[genotype_col].unique()
    alleles = set(''.join(genotypes))  # Например, из 'CC', 'CT', 'TT' получаем {'C', 'T'}

    # Собираем общие подсчеты по аллелям
    allele_counts = {allele: {'with_feature': 0, 'without_feature': 0} for allele in alleles}

    for genotype in genotypes:
        for allele in alleles:
            if allele in genotype:
                allele_count = genotype.count(allele)
                allele_counts[allele]['with_feature'] += allele_count * genetic_data[(genetic_data[genotype_col] == genotype) & (genetic_data[feature_col] == True)].shape[0]
                allele_counts[allele]['without_feature'] += allele_count * genetic_data[(genetic_data[genotype_col] == genotype) & (genetic_data[feature_col] == False)].shape[0]

    # Формируем таблицу сопряженности из подсчетов аллелей
    contingency_table = np.array([list(counts.values()) for _, counts in allele_counts.items()])
    
    # Расчет процентов для каждой ячейки таблицы
    row_sums = contingency_table.sum(axis=1, keepdims=True)
    percentages = (contingency_table / row_sums) * 100

    # Расчет Хи-квадрат и p-значения
    chi2, p, _, _ = chi2_contingency(contingency_table, correction=False)
    chi2 = round(chi2, 2)
    p = round(p, 3)

    # Расчет OR и 95% CI для каждого аллеля
# Расчет OR и 95% CI для каждого аллеля
    allele_statistics = []
    for i, (allele, counts) in enumerate(allele_counts.items()):
        # Для первого аллеля используем таблицу как есть, для второго - обратную
        if i == 0:
            contingency_table = contingency_table
        else:
            contingency_table = contingency_table[::-1]

        or_value, ci_or = odds_ratio_ci(contingency_table)
        allele_statistics.append({
            'Allele': allele,
            'With_Feature': counts['with_feature'],
            'With_Feature_Percent': round(counts['with_feature'] / (contingency_table[0,0] + contingency_table[1,0])*100,2), 
            'Without_Feature': counts['without_feature'],
            'Without_Feature_Percent': round(counts['without_feature'] / (contingency_table[0,1] + contingency_table[1,1])*100,2),
            'Chi2': chi2,
            'p-value': p,
            'Odds Ratio': or_value,
            '95% CI': ci_or
        })


    # Создание DataFrame из результатов
    allele_results_df = pd.DataFrame(allele_statistics)
    
    return allele_results_df


In [82]:
def create_description(statistics_df, gene_name, feature_name, is_allele=False):
    descriptions = []
    for _, row in statistics_df.iterrows():
        identifier = row['Allele'] if is_allele else row['Genotype']
        with_feature_percent = row['With_Feature_Percent']
        without_feature_percent = row['Without_Feature_Percent']
        p_value = row['p-value']
        orf = row['Odds Ratio']
        ci = round_confidence_interval(row['95% CI'])
        
        # Текст описания в зависимости от того, работаем ли мы с генотипом или аллелем
        entity_type = "аллель" if is_allele else "генотип"
        description = (
            f"В группе больных с признаком {feature_name} {entity_type} {identifier} полиморфного локуса {gene_name} "
            f"встречается с частотой {with_feature_percent:.2f}%, что "
        )
        if p_value < 0.05:
            description += (
                f"статистически значимо чаще (p={p_value:.3f}), чем в группе без признака, "
                f"где этот {entity_type} встречается с частотой {without_feature_percent:.2f}%. "
                f"Отношение шансов для {entity_type}а составляет: {orf}."
            )
        else:
            description += (
                f"статистически не отличается (p={p_value:.3f}) от группы без признака, "
                f"где этот {entity_type} встречается с частотой {without_feature_percent:.2f}%. "
                f"Отношение шансов для {entity_type}а составляет: {orf}."
            )
        descriptions.append(description)
    return " ".join(descriptions)


In [83]:
doc = Document()

# Список всех признаков для анализа
features = [
    'Вегетативно-сосудистая_полинейропатия', 'Пояснично-крестцовая радикулопатия', 
    'Периферический ангиодистонический синдром', 'Шейно-плечевая радикулопатия', 
    'Гипертоническая болезнь', 'Ишемическая болезнь', 'ЦВЗ', 
    # 'Остеоартроз', 
    'БА', 'ХОБЛ', 'Нейросенсорная тугоухость', 'СД', 'Пептическая язва'
]

# Список генов для анализа
genes = ['rs4880 гена SOD2', 'rs361525 гена TNFA', 'rs16944 гена IL1B', 'rs1799750 гена MMP1', 'rs1800795 гена IL6']
all_features_results = {}
for gene in genes:
    gene_results = {}
    for feature in features:
        feature_statistics = calculate_extended_statistics(genetic_data, gene, feature)
        gene_results[feature] = feature_statistics
    all_features_results[gene] = gene_results

for feature in features:
    for gene in genes:
        statistics_df = all_features_results[gene][feature]

        for col in ['With_Feature', 'Without_Feature', 'With_Feature_Percent', 'Without_Feature_Percent']:
            statistics_df[col] = statistics_df[col].apply(lambda x: round(x, 2) if isinstance(x, (int, float)) else x)

        statistics_df['p-value'] = statistics_df['p-value'].apply(lambda x: round(x, 3) if isinstance(x, (int, float)) else x)
        statistics_df['95% CI'] = statistics_df['95% CI'].apply(round_confidence_interval)


        doc.add_paragraph(f"Таблица 1. \n Частоты генотипов и аллелей полиморфного локуса {gene} в группах с {feature} и с отсуствием данного признака ")
                # Добавляем заголовок перед описанием

        # Второй шаг - расчет статистики для аллелей
        allele_statistics = calculate_allele_statistics(genetic_data, gene, feature)
        allele_descriptions = create_description(allele_statistics, gene,feature, is_allele=True)

       
        allele_statistics.rename(columns={'Allele': 'Genotype'}, inplace=True)
        combined_df = pd.concat([statistics_df, allele_statistics], ignore_index=True)

        table = doc.add_table(rows=1, cols=len(combined_df.columns))
        table.style = 'Table Grid'
        tbl_borders = {
            "sz": 6, "val": "single", "color": "000000", "space": 1
        }
        tbl_xml = parse_xml(r'<w:tblPr {}><w:tblBorders><w:top {}/><w:left {}/><w:bottom {}/><w:right {}/><w:insideH {}/><w:insideV {}/></w:tblBorders></w:tblPr>'.format(
            nsdecls('w'), *[f'{k}="{v}"' for k, v in tbl_borders.items()] * 6))
        table._element.insert(0, tbl_xml)

        hdr_cells = table.rows[0].cells
        hdr_titles = ["Генотип/Аллель", f"Наличие {feature}", "Процент", f"Отсутствие {feature}", "Процент", "Х2", "p", "OR", "CI95%"]
        for i, title in enumerate(hdr_titles):
            hdr_cells[i].text = title

        for _, row in combined_df.iterrows():
            row_cells = table.add_row().cells
            for i, value in enumerate(row):
                row_cells[i].text = str(value)

        doc.add_paragraph()

        doc.add_paragraph(f"Перепиши в научном стиле. используя красивые и умные слова. и используй разнообразный стиль изложения, чтобы было не шаблонно. не меняй цифры. в конце дай небольшую интерпритацию: ")
        description = create_description(statistics_df, gene, feature)
        doc.add_paragraph(description)
        # Для аллелей

        doc.add_paragraph(allele_descriptions)

try:
    docx_path = 'genetic_analysis_report.docx'
    doc.save(docx_path)
    success = True
except Exception as e:
    error_message = str(e)
    success = False
    docx_path = None

if success:
    print(f"Документ успешно сохранен как: {docx_path}")
else:
    print(f"Ошибка при сохранении документа: {error_message}")


C:\Users\KarimovDO\AppData\Local\Temp\ipykernel_3504\3024606394.py:6: RuntimeWarning: divide by zero encountered in longlong_scalars
  se_log_or = np.sqrt(1/table[0, 0] + 1/table[0, 1] + 1/table[1, 0] + 1/table[1, 1])
C:\Users\KarimovDO\AppData\Local\Temp\ipykernel_3504\3024606394.py:9: RuntimeWarning: divide by zero encountered in log
  ci_log_or = [np.log(or_value) - se_log_or * 1.96, np.log(or_value) + se_log_or * 1.96]
C:\Users\KarimovDO\AppData\Local\Temp\ipykernel_3504\3024606394.py:9: RuntimeWarning: invalid value encountered in double_scalars
  ci_log_or = [np.log(or_value) - se_log_or * 1.96, np.log(or_value) + se_log_or * 1.96]
C:\Users\KarimovDO\AppData\Local\Temp\ipykernel_3504\3024606394.py:3: RuntimeWarning: divide by zero encountered in longlong_scalars
  or_value = round((table[0, 0] * table[1, 1]) / (table[0, 1] * table[1, 0]), 2)
C:\Users\KarimovDO\AppData\Local\Temp\ipykernel_3504\3024606394.py:6: RuntimeWarning: divide by zero encountered in longlong_scalars
  se_lo

Документ успешно сохранен как: genetic_analysis_report.docx


C:\Users\KarimovDO\AppData\Local\Temp\ipykernel_3504\3024606394.py:6: RuntimeWarning: divide by zero encountered in long_scalars
  se_log_or = np.sqrt(1/table[0, 0] + 1/table[0, 1] + 1/table[1, 0] + 1/table[1, 1])
C:\Users\KarimovDO\AppData\Local\Temp\ipykernel_3504\3024606394.py:9: RuntimeWarning: divide by zero encountered in log
  ci_log_or = [np.log(or_value) - se_log_or * 1.96, np.log(or_value) + se_log_or * 1.96]
C:\Users\KarimovDO\AppData\Local\Temp\ipykernel_3504\3024606394.py:9: RuntimeWarning: invalid value encountered in double_scalars
  ci_log_or = [np.log(or_value) - se_log_or * 1.96, np.log(or_value) + se_log_or * 1.96]
C:\Users\KarimovDO\AppData\Local\Temp\ipykernel_3504\3024606394.py:3: RuntimeWarning: divide by zero encountered in long_scalars
  or_value = round((table[0, 0] * table[1, 1]) / (table[0, 1] * table[1, 0]), 2)


In [84]:
def calculate_extended_statistics(genetic_data, genotype_col, feature_col):
    # Получаем уникальные генотипы
    genotypes = genetic_data[genotype_col].unique()
    # Уникальные значения предиктора
    feature_values = genetic_data[feature_col].unique()

    # Результаты для каждого генотипа
    results = []

    for genotype in genotypes:
        for feature_value in feature_values:
            # Подсчет случаев с данным генотипом и уровнем предиктора
            count_feature_value = genetic_data[(genetic_data[genotype_col] == genotype) & (genetic_data[feature_col] == feature_value)].shape[0]
            
            # Подсчет случаев с данным генотипом и другими уровнями предиктора
            count_other = genetic_data[(genetic_data[genotype_col] == genotype) & (genetic_data[feature_col] != feature_value)].shape[0]

            # Проценты для каждого уровня предиктора
            total_count_feature = genetic_data[genetic_data[feature_col] == feature_value].shape[0]
            percent_feature_value = round((count_feature_value / total_count_feature) * 100, 2)

            # Создание таблицы сопряженности
            contingency_table = np.array([[count_feature_value, count_other],
                                          [total_count_feature - count_feature_value, 
                                           genetic_data.shape[0] - total_count_feature - count_other]])
            
            # Расчет Хи-квадрат и p-значения
            chi2, p, _, _ = chi2_contingency(contingency_table, correction=True)
            # print(contingency_table)
            chi2 = round(chi2, 2)
            p = round(p, 2)

            # Расчет OR и 95% CI
            or_value, ci_or = odds_ratio_ci(contingency_table)
            
            # Добавление результатов в список
            results.append({
                'Genotype': genotype,
                'Feature_Value': feature_value,
                'With_Feature': count_feature_value,
                'With_Feature_Percent': percent_feature_value,
                'Chi2': chi2,
                'p-value': p,
                'Odds Ratio': or_value,
                '95% CI': ci_or
            })

    # Создание DataFrame из результатов
    results_df = pd.DataFrame(results)
    
    return results_df


def odds_ratio_ci(table, alpha=0.05):
    # Расчет отношения шансов
    or_value = round((table[0, 0] * table[1, 1]) / (table[0, 1] * table[1, 0]), 2)
    
    # Расчет стандартной ошибки логарифма отношения шансов
    se_log_or = np.sqrt(1/table[0, 0] + 1/table[0, 1] + 1/table[1, 0] + 1/table[1, 1])
    
    # Доверительный интервал для логарифма отношения шансов
    ci_log_or = [np.log(or_value) - se_log_or * 1.96, np.log(or_value) + se_log_or * 1.96]
    
    # Возврат к оригинальному масштабу отношения шансов
    ci_or = [round((np.exp(ci_log_or[0])),2), round((np.exp(ci_log_or[1])),2)]
    
    return or_value, ci_or


def describe_row(row):
    genotype = row['Genotype']
    feature_value = row['Feature_Value']
    count = row['With_Feature']
    percent = row['With_Feature_Percent']
    p_value = row['p-value']
    odds_ratio = row['Odds Ratio']
    ci = row['95% CI']

    description = (
        f"Перепиши в научном стиле. используя красивые и умные слова. и используй разнообразный стиль изложения, чтобы было не шаблонно. не меняй цифры. в конце дай небольшую интерпритацию: Количество пациентов с генотипом {genotype} и тяжестью заболевания {feature_value} было {count} индивидов, "
        f"что составило {percent}% всех пациентов с этой тяжестью заболевания. "
    )

    if p_value < 0.05:
        description += f"Это статистически значимо отличается от противоположной группы с другой тяжестью заболевания (p={p_value}), "
    else:
        description += f"Что статистически не отличается от противоположной группы с другой тяжестью заболевания (p={p_value}), "

    description += f"отношение шансов составило OR={odds_ratio} {ci}."

    return description


def export_to_docx(all_features_results, filename="combined_analysis_report.docx"):
    # Создаем новый документ
    doc = Document()

    # Добавляем заголовок
    doc.add_heading('Общий Генетический Анализ', level=1)

    for gene, features_results in all_features_results.items():
        for feature, dataframe in features_results.items():
            # Добавляем подзаголовок для гена и характеристики
            doc.add_heading(f'{gene} - {feature}', level=2)

            # Добавляем таблицу с результатами
            table = doc.add_table(rows=1, cols=dataframe.shape[1])
            table.style = 'Table Grid'

            # Заполняем заголовки таблицы
            hdr_cells = table.rows[0].cells
            for i, col_name in enumerate(dataframe.columns):
                hdr_cells[i].text = col_name

            # Добавляем строки таблицы
            for index, row in dataframe.iterrows():
                row_cells = table.add_row().cells
                for i, value in enumerate(row):
                    row_cells[i].text = str(value)

            # Добавляем описания
            descriptions = dataframe.apply(describe_row, axis=1)
            for desc in descriptions:
                doc.add_paragraph(desc)

            # Добавляем разрыв страницы перед следующей комбинацией ген-характеристика
            doc.add_page_break()

    # Сохраняем документ
    doc.save(filename)


features = [
    'Тяжесть заболевание'
]
# Список генов для анализа
genes = ['rs4880 гена SOD2', 'rs361525 гена TNFA', 'rs16944 гена IL1B', 'rs1799750 гена MMP1', 'rs1800795 гена IL6']
all_features_results = {}
for gene in genes:
    gene_results = {}
    for feature in features:
        feature_statistics = calculate_extended_statistics(genetic_data, gene, feature)
        gene_results[feature] = feature_statistics
    all_features_results[gene] = gene_results
    # Применение функции к каждой строке DataFrame



# Пример использования
export_to_docx(all_features_results)




C:\Users\KarimovDO\AppData\Local\Temp\ipykernel_3504\1425290231.py:59: RuntimeWarning: divide by zero encountered in long_scalars
  se_log_or = np.sqrt(1/table[0, 0] + 1/table[0, 1] + 1/table[1, 0] + 1/table[1, 1])
C:\Users\KarimovDO\AppData\Local\Temp\ipykernel_3504\1425290231.py:62: RuntimeWarning: divide by zero encountered in log
  ci_log_or = [np.log(or_value) - se_log_or * 1.96, np.log(or_value) + se_log_or * 1.96]
C:\Users\KarimovDO\AppData\Local\Temp\ipykernel_3504\1425290231.py:62: RuntimeWarning: invalid value encountered in double_scalars
  ci_log_or = [np.log(or_value) - se_log_or * 1.96, np.log(or_value) + se_log_or * 1.96]


In [85]:
import pandas as pd
import scipy.stats as stats

# Предположим, что у вас есть DataFrame с данными
# data = ...

genes = ['rs4880 гена SOD2', 'rs361525 гена TNFA', 'rs16944 гена IL1B', 'rs1799750 гена MMP1', 'rs1800795 гена IL6']
parameters = ['Лейкоциты', 'Эритроциты', 'СОЭ', 'САД', 'ДАД', 'ЧСС']

# Функция для расчета статистики по генотипу и аллелю
def calculate_statistics(data, gene, parameter):
    results = []

    # Статистика по генотипам
    for genotype in data[gene].unique():
        subset = data[data[gene] == genotype]
        mean = subset[parameter].mean()
        std = subset[parameter].std()
        n = len(subset)

        results.append({'Type': 'Генотип', 'Category': genotype, 'N': n, 'Среднее': f'{mean:.1f}±{std:.1f}'})
    
    # ANOVA тест
    f_val, p_val = stats.f_oneway(*(data[data[gene] == g][parameter] for g in data[gene].unique()))
    results[-1]['F'] = round(f_val, 3)
    results[-1]['p-value'] = round(p_val, 3)

    # Статистика по аллелям
    for allele in set(''.join(data[gene].unique())):
        subset = data[data[gene].str.contains(allele)]
        mean = subset[parameter].mean()
        std = subset[parameter].std()
        n = len(subset)

        results.append({'Type': 'Аллель', 'Category': allele, 'N': n, 'Среднее': f'{mean:.1f}±{std:.1f}'})
    
    # t-тест
    t_val, p_val = stats.ttest_ind(data[data[gene].str.contains(results[-2]['Category'])][parameter],
                                   data[data[gene].str.contains(results[-1]['Category'])][parameter])
    results[-1]['t'] = round(t_val, 3)
    results[-1]['p-value'] = round(p_val, 3)

    return results

def describe_statistics(statistics, parameter):
    descriptions = []

    for item in statistics:
        if item['Type'] == 'Генотип':
            description = f"Среднее количество показателя {parameter} в группе индивидов с генотипом {item['Category']} составило {item['Среднее']}"
            descriptions.append(description)
        
        if 'F' in item:
            significance = "не значимые" if item['p-value'] >= 0.05 else "достигли уровня статистической значимости"
            descriptions.append(f"Различия между группами были {significance} (F = {item['F']}, p= {item['p-value']}).")

    for item in statistics:
        if item['Type'] == 'Аллель':
            description = f"В группе с аллелем {item['Category']} среднее значение {parameter} составило {item['Среднее']}"
            descriptions.append(description)
        
        if 't' in item:
            significance = "не значимо" if item['p-value'] >= 0.05 else "значимо"
            direction = "ниже" if statistics[-2]['Среднее'] > statistics[-1]['Среднее'] else "выше"
            descriptions.append(f"При сравнении средних значений показателя {parameter}, мы обнаружили, что в группе с аллелем {item['Category']} оно было статистически {significance} {direction} (t = {item['t']}, p = {item['p-value']})")

    return ' '.join(descriptions)

def create_tables(statistics):
    # Создаем отдельные списки для генотипов и аллелей
    genotype_data = [item for item in statistics if item['Type'] == 'Генотип']
    allele_data = [item for item in statistics if item['Type'] == 'Аллель']

    # Создаем DataFrame для генотипов
    genotype_df = pd.DataFrame(genotype_data)
    genotype_df = genotype_df.drop(['Type'], axis=1)

    # Создаем DataFrame для аллелей
    allele_df = pd.DataFrame(allele_data)
    allele_df = allele_df.drop(['Type'], axis=1)

    return genotype_df, allele_df


# Пример использования
all_stats = {}
for gene in genes:
    gene_stats = {}
    for parameter in parameters:
        gene_stats[parameter] = calculate_statistics(genetic_data, gene, parameter)
    all_stats[gene] = gene_stats

from docx import Document
from docx.shared import Pt
from docx.oxml.ns import qn
from docx.enum.text import WD_ALIGN_PARAGRAPH
import copy

def save_to_docx(all_stats, filename="genetic_analysis_report_mean.docx"):
    # Создаем документ
    doc = Document()

    for gene in genes:
        for parameter in parameters:
            statistics = all_stats[gene][parameter]
            genotype_df, allele_df = create_tables(statistics)
            description = describe_statistics(statistics, parameter)

            # Добавляем подзаголовок для таблицы генотипов
            doc.add_paragraph(f"Таблица 1. Среднее значение параметра {parameter} и статистическая значимость различий в зависимости от генотипа и наличия аллелей полиморфного локуса {gene}:", style='Caption')

            # Добавляем таблицу генотипов
            add_table_to_doc(doc, genotype_df)

            # Добавляем таблицу аллелей
            add_table_to_doc(doc, allele_df)

            # Добавляем описание
            p = doc.add_paragraph()
            p.add_run("Перепиши в научном стиле. используя красивые и умные слова. и используй разнообразный стиль изложения, чтобы было не шаблонно. не меняй цифры. в конце дай небольшую интерпритацию: ").bold = True
            p.add_run(description)

            # Добавляем разделитель между разными комбинациями
            doc.add_paragraph("\n" + "-"*50 + "\n")

    # Сохраняем документ
    doc.save(filename)

def add_table_to_doc(doc, df):
    table = doc.add_table(rows=1, cols=len(df.columns))
    table.style = 'Table Grid'

    # Заполняем заголовки таблицы
    hdr_cells = table.rows[0].cells
    for i, col_name in enumerate(df.columns):
        hdr_cells[i].text = col_name

    # Добавляем строки таблицы
    for index, row in df.iterrows():
        row_cells = table.add_row().cells
        for i, value in enumerate(row):
            row_cells[i].text = str(value)

    # Объединение ячеек
    merge_cells(table, 1, 3, 4)  # Объединение ячеек в столбце 'F'
    merge_cells(table, 1, 4, 4)  # Объединение ячеек в столбце 'p-value' для генотипов
    merge_cells(table, 5, 3, 6)  # Объединение ячеек в столбце 't' для аллелей
    merge_cells(table, 5, 4, 6)  # Объединение ячеек в столбце 'p-value' для аллелей

def merge_cells(table, start_row, col_idx, end_row):
    num_rows = len(table.rows)
    # Убедимся, что start_row и end_row находятся в пределах таблицы
    start_row = max(1, min(start_row, num_rows - 1))
    end_row = min(end_row, num_rows)
    table_copy = copy.deepcopy(table)
    # Объединяем ячейки
    cell_to_merge = table.cell(start_row, col_idx)
    for i in range(start_row + 1, end_row):
        cell = table.cell(i, col_idx)
        cell_to_merge.merge(cell)
        # Устанавливаем текст из последней ячейки объединения
    cell_to_merge.text = table_copy.cell(end_row - 1, col_idx).text

# Пример использования
save_to_docx(all_stats)